### Simple RAG (Retrieval-Augmented generation) System for CSV files

### Overview

###### This code implements a basic Retrieval-Augmented Generation (RAG) system for processing and querying CSV documents. The system encodes the document content into a vector store, which can then be queries to retrieve relevant information.

### CSV file Structure and Use Case

###### The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

### Key Components

###### 1. Loading and Splitting csv files.
###### 2. Vector store creation using FAISS and OpenAI embeddings.
###### 3. Query engine setup for querying the processed documents.
###### 4. Creating a question nd answer over the csv data.

### Method Details

#### Document Preprocessing

###### 1. The csv is loaded using LlamaIndex's PagedCSVReader
###### 2. A FAISS vector store is created from these embeddings for efficient similarity search

### Query Engine Setup

###### 1. A query engine is configured to fetch the most relevant chunks for a given query then answer the questions

### Benefits of this Approach

###### 1. Scalability: Can handle large documents by processing them in chunks
###### 2. Flexibility: Easy to adjust parameters like chunk size and number of retrieved results
###### 3. Efficiency: Utilizes FAISS for fast similarity search in high-dimensional spaces.
###### 4. Integration with Advanced NLP: Uses OpenAI embeddings for the state-of-the-art text representation.

### Conclusion

###### This simple RAG system provides a solid foundation for building more complex information retrieval and question-answering systems. By encoding document content into a searchable vector store, it enables efficient retrieval of relevant information in response to queries. This approach is particularly useful for applications requiring quick access to specific information within a CSV file

### Import & Environment Variables


In [ ]:
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import PagedCSVReader
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
import faiss
import os
import pandas as pd
from dotenv import load_dotenv

#Load environment variables from a .env file
load_dotenv()

#Set the OpenAI API key environment variable
os.environ["OpenAI_API_KEY"]=os.getenv('OpenAI_API_KEY')

#llama_index global Settings for llm and embeddings
EMBED_DIMENSION=512
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small", dimensions=EMBED_DIMENSION)

### CSV File Structure and Use Case

###### The CSV file contains dummy customer data, comprising values attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

In [ ]:
file_path = ('../data/customers-100.csv') # Insert the path of the csv file
data = pd.read_csv(file_path)

#Preview the csv file
data.head()

### Vector Store

In [ ]:
# Create FaisVectorStore to store embeddings

fais_index = fais.IndexFlatL2(EMBED_DIMENSION)
vector_store = FaisVectorStore(fais_index=fais_index)

### Load and Process CSV Data as Document

In [ ]:
csv_reader = PagedCSVReader()

reader = SimpleDirectoryReader(
    input_files = [file_path],
    file_extractor={".csv": csv_reader}
)

docs = reader.load_data()

In [ ]:
#Check a simple chunk

print(docs[0].text)

### Ingestion Pipeline

In [ ]:
pipeline = IngestionPipeline(
    vector_store=vector_store,
    documents=docs
)

nodes = pipeline.run()

### Create Query Engine

In [ ]:
vector_store_index = VectorStoreIndex(nodes)
query_engine = vector_store_index.as_query_engine(similarity_top_k=2)

### Query the rag bot with a question based on the CSV data

In [ ]:
response = query_engine.query("Which company does Sheryl Baxter work for?")
response.response